In [ ]:

# coding: utf-8
from __future__ import print_function
import os
import numpy as np
np.random.seed(123)

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline 
import seaborn as sns
import pandas as pd
import numpy as np
import sys

BASE_DIR = '../../'
GLOVE_DIR = BASE_DIR + 'glove/'


svc_tfidf = Pipeline([
    ("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), 
    ("linear svc", SVC(kernel="linear"))
])
doc_id = 161
author_id = 80
authorList = [11, 18, 80, 88, 64]
chunk_size = 1500

In [ ]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
import DatabaseQuery
# textToUse = pd.read_csv("suffle_4_6000.csv", names=["author_id", "doc_content"], dtype={'author_id': int})
from sshtunnel import SSHTunnelForwarder
PORT=5432
with SSHTunnelForwarder(('srn02.cs.cityu.edu.hk', 22),
                        ssh_username='stylometry',
                        ssh_password='stylometry',
                        remote_bind_address=('localhost', 5432),
                        local_bind_address=('localhost', 5400)):
    textToUse = DatabaseQuery.getWordAuthData(5400, authors = authorList, doc = doc_id,
                                              chunk_size = chunk_size)

In [ ]:
textToUse.loc[1000]

In [ ]:
labels = []
texts = []
size = []
authorList = textToUse.author_id.unique()
for auth in authorList:
    current = textToUse.loc[textToUse['author_id'] == auth]
    size.append(current.shape[0])
    print("Author: %5s  Size: %5s" % (auth, current.shape[0]))
print("Min: %s" % (min(size)))
print("Max: %s" % (max(size)))

authorList = authorList.tolist()

for auth in authorList:
    current = textToUse.loc[textToUse['author_id'] == auth]
    samples = min(size)
    current = current.sample(n = samples)
    textlist = current.doc_content.tolist()
    texts = texts + textlist
    labels = labels + [authorList.index(author_id) for author_id in current.author_id.tolist()]

labels_index = {}
labels_index[0] = 0
for i, auth in enumerate(authorList):
    labels_index[i] = auth

del textToUse

In [ ]:
# split the data into a training set and a validation set
# from sklearn.model_selection import KFold
# kfold = KFold(n_splits=6, shuffle=True, random_state=123)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2)

In [ ]:

svc_tfidf.fit(x_train, y_train)


In [ ]:
print('Train Accuracy %s.' % (str(svc_tfidf.score(x_train, y_train))))
print('Test Accuracy %s.' % (str(svc_tfidf.score(x_val, y_val))))

In [ ]:
texts = []  # list of text samples
labels = []  # list of label ids
import DatabaseQuery
# textToUse = pd.read_csv("suffle_4_6000.csv", names=["author_id", "doc_content"], dtype={'author_id': int})
from sshtunnel import SSHTunnelForwarder
PORT=5432
with SSHTunnelForwarder(('srn02.cs.cityu.edu.hk', 22),
                        ssh_username='stylometry',
                        ssh_password='stylometry',
                        remote_bind_address=('localhost', 5432),
                        local_bind_address=('localhost', 5400)):
    textToUse = DatabaseQuery.getWordDocData(5400, doc_id, chunk_size = chunk_size)
labels = []
texts = []
for index, row in textToUse.iterrows():
    labels.append(authorList.index(row.author_id))
    texts.append(row.doc_content)
        
print('Found %s texts.' % len(texts))

del textToUse


In [ ]:
svc_tfidf.score(texts, labels)

In [ ]:
predList = svc_tfidf.predict(texts)

In [ ]:
predList

In [ ]:
predY = []
for i, val in enumerate(authorList):
    pred = (float)(list(predList).count(i)) / len(predList)
    predY.append(pred * )

In [ ]:
for i, val in enumerate(predY):
    print("%s  %s" % (str(i), str(val)))

In [ ]:
labels_index